Econometrics II PS 3
==
Maximilian Huber


You can find this code on my GitHub account: https://github.com/MaximilianJHuber/NYU
1.
==
a.
--

In [1]:
using Optim
using DataFrames
using ForwardDiff
using Roots
using Distributions

LoadError: ArgumentError: module Optim not found in current path.
Run `Pkg.add("Optim")` to install the Optim package.

In [2]:
data = convert(Matrix,readtable("hsdata.csv"));
dataexp = exp(data);

LoadError: UndefVarError: readtable not defined

In [3]:
#operates on single observations!
function g(w,z,theta) 
    return (theta[1] * w[1]^(-theta[2]) * w[2] - 1) .* collect(z[:])
end

#operates on data set!
function gn(w,z,theta)
    i = 1
    result = g(w[i,:],z[i,:],theta)
    
    for i in 2:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)
    end
    return result/size(w)[1]
end

#Wrapper creates a closure around the data set provided
function Qn_wrapper(w,z)
    return theta -> 1/2*(gn(w,z,theta)'*gn(w,z,theta))[1,1]
end

Qn_wrapper (generic function with 1 method)

In [4]:
w = dataexp[2:end,[2,4]]
z = hcat(ones(size(dataexp)[1]-1), dataexp[1:end-1,4], 1 .+ data[1:end-1,2])

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.], BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true))

LoadError: UndefVarError: dataexp not defined

So $\theta_A = (0.995,4.4399)$.

b.
--
I define:
$$\hat{W}_A=\left(\frac{1}{n} \sum_{t=1}^n{g(w_t,\theta_A)g(w_t,\theta_A)'}\right)^{-1}$$
as in equation (3.5).

In [5]:
thetaA = optres.minimum

function What(w,z,theta) 
    result = zeros(Float64, size(z)[2], size(z)[2])
    for i in 1:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)*g(w[i,:],z[i,:],theta)'
    end
    return inv(result/size(z)[1])
end

WhatA = What(w,z,thetaA) 

LoadError: UndefVarError: optres not defined

In [6]:
function Qn_wrapper(w,z,What)
    return theta -> 1/2*(gn(w,z,theta)'*What*gn(w,z,theta))[1,1]
end

QnWhatA = Qn_wrapper(w,z,WhatA)
optresB = optimize(QnWhatA, thetaA, BFGS(), Optim.OptimizationOptions(g_tol=1e-12))

LoadError: UndefVarError: w not defined

So $\theta_B = (0.9785,3.3860)$.

c.
--

I need an estimate for the asymptotic variance $\left(G'S^{-1}G\right)^{-1}$:

$$\hat{G} = \frac{1}{n} \sum_{t=1}^n{\frac{\partial d(w_t,\theta_B)}{\partial \theta}} = \frac{\partial \frac{1}{n} \sum_{t=1}^n{d(w_t,\theta_B)}}{\partial \theta} = \frac{\partial gn(\theta_B)}{\partial \theta}$$ and 
$$\hat{S} = \hat{W}_B$$

I use automatic differentiation (rather than numeric differentiation) for all calculations (see, http://www.juliadiff.org/).

In [7]:
thetaB = optresB.minimum

#creats a closure around the data set
function gn_wrapper(w,z)
    return theta -> gn(w,z,theta)
end

dgn(theta) = ForwardDiff.jacobian(gn_wrapper(w,z),theta)

GhatB = convert(Array{Float64,2},dgn(thetaB))

LoadError: UndefVarError: optresB not defined

In [8]:
Shat = inv(What(w,z,thetaB))

LoadError: UndefVarError: What not defined

In [9]:
aVar = inv(GhatB'*inv(Shat)*GhatB)

LoadError: UndefVarError: GhatB not defined

In [10]:
thetaB[1] .+ sqrt(aVar[1,1]) * [-1.96, 1.96]

LoadError: UndefVarError: thetaB not defined

In [11]:
thetaB[2] .+ sqrt(aVar[2,2]) * [-1.96, 1.96]

LoadError: UndefVarError: thetaB not defined

d.
--
Since I do minimization of a positive function, the Hansen-Sargan statistic is:
$$J=2Qn(\theta_B)$$

In [12]:
2*size(z)[1]*QnWhatA(thetaB)

LoadError: UndefVarError: z not defined

In [13]:
1-cdf(Chisq(1),2*size(z)[1]*QnWhatA(thetaB))

LoadError: UndefVarError: cdf not defined

The p-value is not significant. I accept the $H_0$, that the $\mathbb{E}\left[ g(w_t;\theta_0 \right]=0$.
e.
--

In [14]:
w = dataexp[3:end,[2,4]]
z = hcat(ones(size(dataexp)[1]-2), dataexp[2:end-1,4], 1 .+ data[2:end-1,3], 1 .+ data[2:end-1,2],
        dataexp[1:end-2,4], 1 .+ data[1:end-2,3], 1 .+ data[1:end-2,2])

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.], BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true))

LoadError: UndefVarError: dataexp not defined

In [15]:
thetaA = optres.minimum
WhatA = What(w,z,thetaA) 

LoadError: UndefVarError: optres not defined

In [16]:
QnWhatA = Qn_wrapper(w,z,WhatA)
optresB = optimize(QnWhatA, thetaA, BFGS(), Optim.OptimizationOptions(g_tol=1e-12))

LoadError: UndefVarError: w not defined

In [17]:
thetaB = optresB.minimum
dgn(theta) = ForwardDiff.jacobian(gn_wrapper(w,z),theta)
GhatB = convert(Array{Float64,2},dgn(thetaB))
Shat = inv(What(w,z,thetaB))
aVar = inv(GhatB'*inv(Shat)*GhatB)
thetaB[1] .+ sqrt(aVar[1,1]) * [-1.96, 1.96]

LoadError: UndefVarError: optresB not defined

In [18]:
thetaB[2] .+ sqrt(aVar[2,2]) * [-1.96, 1.96]

LoadError: UndefVarError: thetaB not defined

In [19]:
2*size(z)[1]*QnWhatA(thetaB)

LoadError: UndefVarError: z not defined

In [20]:
1-cdf(Chisq(1),2*size(z)[1]*QnWhatA(thetaB))

LoadError: UndefVarError: cdf not defined

2.
==
a.
--
With 
$
\Gamma=\left(\begin{array}{cc}
1 & 0\\
-\gamma & 1
\end{array}\right)^{-1}
$, $\Sigma=\left(\begin{array}{cc}
\sigma_{c}^{2} & \rho\sigma_{c}\sigma_{r}\\
\rho\sigma_{c}\sigma_{r} & \sigma_{r}^{2}
\end{array}\right)$ and $\sigma^{2}(\gamma)=\left(\begin{array}{cc}
-\gamma & 1\end{array}\right)\Sigma\left(\begin{array}{c}
-\gamma\\
1
\end{array}\right)$ I find:

$$x_{t+1}=\left(\begin{array}{c}
c_{t+1}\\
r_{t+1}
\end{array}\right)-\Gamma\left(\begin{array}{cc}
\alpha & \beta\\
0 & 0
\end{array}\right)\left(\begin{array}{c}
c_{t}\\
r_{t}
\end{array}\right)-\Gamma\left(\begin{array}{c}
\mu_{c}\\
\lg(\delta)+\frac{1}{2}\sigma^{2}(\gamma)
\end{array}\right)\sim\mathbb{\mathcal{N}}\left(\left[\begin{array}{c}
0\\
0
\end{array}\right],\Gamma\Sigma\Gamma'\right)$$
I maximize: $$Q_{n}\left(\theta\right)=\frac{1}{n}\sum_{t=1}^{n-1}\lg\: f\left(x_{t+1}\mid x_{t};\theta\right)$$

In [21]:
w = data[1:end,[2,4]]

#theta = (δ,γ,α,β,μc,σc,σr,ρ)

#for a single observation:
@inline function m(w,wold,theta)
    (δ,γ,α,β,μc,σc,σr,ρ) = theta
    Σ = [σc^2 ρ*σc*σr; ρ*σc*σr σr^2]
    Γ = inv([1 0; -γ 1])
    
    x = w[:] - (Γ*[α β; 0 0]*wold[:]) - Γ*[μc; log(δ) + [-γ 1]*Σ*[-γ; 1]]
    return log(pdf(MvNormal([0, 0], Γ*Σ*Γ'),x))
end

function Qn_wrapper(w)
    return function(theta)
        i = 2
        result = m(w[i,:],w[i-1,:],theta)
        
        for i in 3:size(w)[1]
            result += m(w[i,:],w[i-1,:],theta)
        end
        
        return -result/size(w)[1] #I am going to minimize this function!
    
        end
end

LoadError: UndefVarError: data not defined

In [22]:
Qn = Qn_wrapper(w)
#theta =     (δ, γ , α,   β,  μc, σc, σr, ρ)

optres = optimize(Qn, [1,0.1,0.5,0.99,1.1,0.8,0.4,0.5], 
            NelderMead(), Optim.OptimizationOptions(iterations=5000, autodiff=true))

LoadError: UndefVarError: w not defined

In [23]:
thetahat = optres.minimum

LoadError: UndefVarError: optres not defined

Since I am worried that the model is misspecified I calculate the sandwich estimator for the asymptotic variance $\hat{H}^{-1} \hat{\Sigma} \hat{H}^{-1}$ where:

$$\hat{H}=\frac{1}{n} \sum_{t=1}^{n-1} {H(w_t; \hat{\theta}}) = \frac{\partial^2 Q_n(\hat{\theta})}{\partial \theta \partial \theta'}$$

$$\hat{\Sigma} = \frac{1}{n} \sum_{t=1}^{n-1} {\frac{\partial \lg\: f\left(x_{t+1}\mid x_{t};\hat{\theta}\right) }{\partial \theta} \, \frac{\partial \lg\: f\left(x_{t+1}\mid x_{t};\hat{\theta}\right) }{\partial \theta}' }$$

In [24]:
Hhat = ForwardDiff.hessian(Qn,thetahat)

LoadError: UndefVarError: ForwardDiff not defined

In [25]:
function Sigmahat(w,theta) 
    i = 2
    result = ForwardDiff.gradient(theta -> m(w[i,:],w[i-1,:],theta), thetahat)*
            ForwardDiff.gradient(theta -> m(w[i,:],w[i-1,:],theta), thetahat)'
    
    for i in 3:size(w)[1]
        result .+= ForwardDiff.gradient(theta -> m(w[i,:],w[i-1,:],theta), thetahat)*
            ForwardDiff.gradient(theta -> m(w[i,:],w[i-1,:],theta), thetahat)'
    end
    return result/size(w)[1]
end

Sigmahat (generic function with 1 method)

In [26]:
Shat = Sigmahat(w,thetahat);

LoadError: UndefVarError: w not defined

In [27]:
aVar = inv(Hhat)*Shat*inv(Hhat)

LoadError: UndefVarError: Hhat not defined

Now the estimated confidece intervals for $\delta$ and $\gamma$:

In [28]:
thetahat[1] .+ sqrt(aVar[1,1]) * [-1.96, 1.96]

LoadError: UndefVarError: thetahat not defined

In [29]:
thetahat[2] .+ sqrt(aVar[2,2]) * [-1.96, 1.96]

LoadError: UndefVarError: thetahat not defined

c.
--
The estimates are less believable in part 2. A $\delta>0$ means that consumers have a negative subjective discount rate and a $\gamma<1$ is also not plausible. But I believe this is because the normality assumption is particularily unwarranted and the linerization of the model leads to certainty equivalence.